In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm
import shutil
import re
import copy
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from pprint import pprint

In [ ]:
adr1 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/us4.csv"
adr2 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/india.csv"
adr3 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/india-reliable.csv"
adr4 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/us4-reliable.csv"


adr_kr_3days = "./DataCollection/Data/Kerala _ 3days/R/"
adr_knr_3days = "./DataCollection/Data/Kerala _ 3days/NR/"


adr_kr_outlets = "./DataCollection/Data/Kerala _ 3days/NR_credibleOutlets/"

In [ ]:
def tweetElements(tweet):
    rtFlag = True if "retweeted_status" in tweet.keys() else False
    qtFlag = True if "quoted_status" in tweet.keys() else False 

    if rtFlag == False and qtFlag == False:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user]
    
    elif rtFlag == True and qtFlag == False:
        rt = copy.deepcopy(tweet["retweeted_status"])
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, rtUser]
    
    elif rtFlag == False and qtFlag == True:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        qt = copy.deepcopy(tweet["quoted_status"])
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser]        
        
    elif rtFlag == True and qtFlag == True:
        rt = copy.deepcopy(tweet["retweeted_status"])
        qt = copy.deepcopy(tweet["quoted_status"])        
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])        
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser, rtUser]        

In [ ]:
df = pd.read_csv(adr3, delimiter=";", skiprows=1 , names=[1,2,3])
df1 = df.drop(columns=[1,2])
df2 = df.drop(columns=[1,3])
df1 = df1.rename({2:"twitter", 3:"site"}, axis="columns")
df2 = df2.rename({2:"twitter", 3:"site"}, axis="columns")

df2["twitter"] = df2["twitter"].str.lower().str.strip()

highlyCredibleWebsitesIndia = [q for q in df1.to_dict(orient="list")["site"] if str(q) != "nan"]
highlyCredibleAccountsIndia = [p for p in df2.to_dict(orient="list")["twitter"] if str(p) != "nan"]

In [ ]:
df = pd.read_csv(adr1, delimiter=";", skiprows=1 , names = [1,2,3,4,5,6,7,8,9], encoding = "ISO-8859-1")
df = df.drop([1,5,6,7,8,9], axis="columns")
df = df.rename({2:"source", 3:"id", 4:"website"}, axis="columns")

df1 = df.drop(columns=["id"], axis="columns")
df2 = df.drop(columns=["website"], axis="columns")

df2["id"] = df2["id"].str.lower().str.replace("?", "").str.strip()

credibleWebsitesUsa = [i for i in df1.to_dict(orient="list")["website"] if str(i) != "nan"]
credibleAccountsUsa = [j for j in df2.to_dict(orient="list")["id"] if str(j) != "nan"]

In [ ]:
df = pd.read_csv(adr2, delimiter=";", skiprows=1 , names=[1,2,3])
df1 = df.drop(columns=[1,2])
df2 = df.drop(columns=[1,3])
df1 = df1.rename({2:"twitter", 3:"site"}, axis="columns")
df2 = df2.rename({2:"twitter", 3:"site"}, axis="columns")

df2["twitter"] = df2["twitter"].str.lower().str.strip()

credibleWebsitesIndia = [q for q in df1.to_dict(orient="list")["site"] if str(q) != "nan"]
credibleAccountsIndia = [p for p in df2.to_dict(orient="list")["twitter"] if str(p) != "nan"]

In [ ]:
credibleAccountsIndia = credibleAccountsIndia + credibleAccountsUsa
credibleWebsitesIndia = credibleWebsitesIndia + credibleWebsitesUsa

In [ ]:
for file in tqdm(os.listdir(adr_kr_3days)):
    qtText, qtEntities, qtUser, rtUser, tweetText, qtTweetText, rtUser = None, None, None, None, None, None, None

    tweet = json.load(open(adr_kr_3days+file))    
    personalViewFlag = True
    
    elements = tweetElements(tweet)
    rtFlag = copy.deepcopy(elements[0])
    qtFlag = copy.deepcopy(elements[1]) 

    
    if rtFlag == False and qtFlag == False:
        text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
    elif rtFlag == True and qtFlag == False:
        text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
    elif rtFlag == False and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
    elif rtFlag == True and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])
                    
    tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
    qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "
            
    urlCredibility = len([i for i in credibleWebsitesIndia if i in entities["urls"][0]["expanded_url"]]) > 0 if len(entities["urls"]) > 0 else False
    profileUrlCredibility = len([i for i in credibleWebsitesIndia if i in user["url"].lower()]) > 0 if user["url"] != None else False
    rtProfileUrlCredibility = (len([i for i in credibleWebsitesIndia if i in rtUser["url"].lower()]) > 0 if rtUser["url"] != None else False) if rtFlag else False
    misinfoFlag = re.search('.*(rumor|rumour|hoax|conspirac|fake-?news|fake-?info| myth | myths |disinfo|misinfo).*', tweetText) != None 
     
    ####################################################### Exception Flag #######################################################
    
    uaeFlag = False
    if re.search('.*fact-?check.*', tweetText, re.DOTALL) != None\
    or re.search('.*(united.*arab.*emirates|uae|emirates|arab region).*(700|100).*(crore|cr|core|million).*', tweetText, re.DOTALL) != None \
    or re.search('.*(700|100).*(crore|cr|core|million).*(united.*arab.*emirates|uae|emirates|arab region).*', tweetText, re.DOTALL) != None \
    or re.search('.*(united.*arab.*emirates|uae|emirates|arab region).*assist.*', tweetText, re.DOTALL) != None \
    or re.search('.*(assist).*(united.*arab.*emirates|uae|emirates|arab region).*', tweetText, re.DOTALL) != None \
    or re.search('.*yusuff.*ali.*', tweetText, re.DOTALL) != None\
    or re.search('.*the viral photos of rss workers.*', tweetText, re.DOTALL) != None\
    or re.search('.*cancel.*onam.*', tweetText, re.DOTALL) != None\
    or re.search('.*onam.*cancel.*', tweetText, re.DOTALL) != None\
    or re.search('.*the richest man in india who lives in a mumbai.*', tweetText, re.DOTALL) != None:
        uaeFlag = True
    else:
        uaeFlag = False
    
    quaeFlag = False
    if re.search('.*fact-?check.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*(united arab emirates|uae|emirates|arab region).*(700|100).*(crore|cr|core|million).*', qtTweetText, re.DOTALL) != None \
    or re.search('.*(700|100).*(crore|cr|core|million).*(united arab emirates|uae|emirates|arab region).*', qtTweetText, re.DOTALL) != None \
    or re.search('.*(united.*arab.*emirates|uae|emirates|arab region).*assist.*', qtTweetText, re.DOTALL) != None \
    or re.search('.*(assist).*(united.*arab.*emirates|uae|emirates|arab region).*', qtTweetText, re.DOTALL) != None \
    or re.search('.*yusuff.*ali.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*the viral photos of rss workers.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*cancel.*onam.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*onam.*cancel.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*the richest man in india who lives in a mumbai.*', qtTweetText, re.DOTALL) != None:
        quaeFlag = True
    else:
        quaeFlag = False

    urlContentCredibility = False
    if len(entities["urls"]) >0:
        if re.search('.*fact-?check.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None\
        or re.search('.*united.*arab.*emirates.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None \
        or re.search('.*700.*(crore|cr).*', entities["urls"][0]["expanded_url"], re.DOTALL) != None\
        or re.search('.*100.*million.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None \
        or re.search('.*yusuff.*ali.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None\
        or re.search('.*(uae|emirates).*', entities["urls"][0]["expanded_url"], re.DOTALL) != None:
            urlContentCredibility = True
        else:
            urlContentCredibility = False
    
    urlContentCredibility = False
    if qtFlag:
        if len(qtEntities["urls"]) >0:
            if re.search('.*fact-?check.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None\
            or re.search('.*united.*arab.*emirates.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None \
            or re.search('.*700.*(crore|cr).*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None\
            or re.search('.*100.*million.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None \
            or re.search('.*yusuff.*ali.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None\
            or re.search('.*(uae|emirates).*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None:
                urlContentCredibility = True
            else:
                urlContentCredibility = False 
    
   
    if uaeFlag or quaeFlag or urlContentCredibility or urlContentCredibility:
        exeptionFlag = False
    else:
        exeptionFlag = True   
    ####################################################### Exception Flag #######################################################

    
    
    ####################################################### Last Dicovery #######################################################

    lastDiscovery = False
    if re.search('.*indian businessmen in uae give 2.7 million.*', tweetText, re.DOTALL) != None\
    or re.search('.*(coconut oil|dubaipolicehq|eidmubarak).*', tweetText, re.DOTALL) != None\
    or re.search('.*700.*', tweetText, re.DOTALL) != None\
    or re.search('.*three business tycoons.*', tweetText, re.DOTALL) != None\
    or re.search('.*donated 1000 dirhams.*', tweetText, re.DOTALL) != None\
    or re.search('.*(united arab emirates|uae|emirates).*', qtTweetText, re.DOTALL) != None:
        lastDiscovery = True
    else:
        lastDiscovery = False  
       
    qlastDiscovery = False
    if re.search('.*indian businessmen in uae give 2.7 million.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*(coconut oil|dubaipolicehq|eidmubarak).*', qtTweetText, re.DOTALL) != None\
    or re.search('.*700.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*three business tycoons.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*donated 1000 dirhams.*', qtTweetText, re.DOTALL) != None\
    or re.search('.*(united arab emirates|uae|emirates).*', qtTweetText, re.DOTALL) != None:
        qlastDiscovery = True
    else:
        qlastDiscovery = False     
        


    ####################################################### Last Dicovery #######################################################
    if (qlastDiscovery or lastDiscovery) and exeptionFlag:
        shutil.move(adr_kr_3days+file, adr_kr_outlets)

In [ ]:
qtText, qtEntities, qtUser, rtUser, tweetText, qtTweetText, rtUser = None, None, None, None, None, None, None

tweet = json.load(open(adr_kr_3days+"237478.json"))    
personalViewFlag = True

elements = tweetElements(tweet)
rtFlag = copy.deepcopy(elements[0])
qtFlag = copy.deepcopy(elements[1]) 


if rtFlag == False and qtFlag == False:
    text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
elif rtFlag == True and qtFlag == False:
    text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
elif rtFlag == False and qtFlag == True:
    text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
elif rtFlag == True and qtFlag == True:
    text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])

tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "

urlCredibility = len([i for i in credibleWebsitesIndia if i in entities["urls"][0]["expanded_url"]]) > 0 if len(entities["urls"]) > 0 else False
profileUrlCredibility = len([i for i in credibleWebsitesIndia if i in user["url"].lower()]) > 0 if user["url"] != None else False
rtProfileUrlCredibility = (len([i for i in credibleWebsitesIndia if i in rtUser["url"].lower()]) > 0 if rtUser["url"] != None else False) if rtFlag else False
misinfoFlag = re.search('.*(rumor|rumour|hoax|conspirac|fake-?news|fake-?info| myth | myths |disinfo|misinfo).*', tweetText) != None 

####################################################### Exception Flag #######################################################
uaeFlag = False
if re.search('.*fact-?check.*', tweetText, re.DOTALL) != None\
or re.search('.*united.*arab.*emirates.*', tweetText, re.DOTALL) != None \
or re.search('.*700.*(crore|cr).*', tweetText, re.DOTALL) != None\
or re.search('.*100.*million.*', tweetText, re.DOTALL) != None \
or re.search('.*yusuff.*ali.*', tweetText, re.DOTALL) != None\
or re.search('.*(uae|emirates).*', tweetText, re.DOTALL) != None\
or re.search('.*the viral photos of rss workers.*', tweetText, re.DOTALL) != None\
or re.search('.*cancel.*onam.*', tweetText, re.DOTALL) != None\
or re.search('.*onam.*cancel.*', tweetText, re.DOTALL) != None\
or re.search('.*the richest man in india who lives in a mumbai.*', tweetText, re.DOTALL) != None:    
    uaeFlag = True
else:
    uaeFlag = False

quaeFlag = False
if re.search('.*fact-?check.*', qtTweetText, re.DOTALL) != None\
or re.search('.*united.*arab.*emirates.*', qtTweetText, re.DOTALL) != None \
or re.search('.*700.*(crore|cr).*', qtTweetText, re.DOTALL) != None\
or re.search('.*100.*million.*', qtTweetText, re.DOTALL) != None \
or re.search('.*yusuff.*ali.*', qtTweetText, re.DOTALL) != None\
or re.search('.*(uae|emirates).*', qtTweetText, re.DOTALL) != None\
or re.search('.*the viral photos of rss workers.*', qtTweetText, re.DOTALL) != None\
or re.search('.*cancel.*onam.*', qtTweetText, re.DOTALL) != None\
or re.search('.*onam.*cancel.*', qtTweetText, re.DOTALL) != None\
or re.search('.*the richest man in india who lives in a mumbai.*', qtTweetText, re.DOTALL) != None:
    quaeFlag = True
else:
    quaeFlag = False

urlContentCredibility = False
if len(entities["urls"]) >0:
    if re.search('.*fact-?check.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None\
    or re.search('.*united.*arab.*emirates.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None \
    or re.search('.*700.*(crore|cr).*', entities["urls"][0]["expanded_url"], re.DOTALL) != None\
    or re.search('.*100.*million.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None \
    or re.search('.*yusuff.*ali.*', entities["urls"][0]["expanded_url"], re.DOTALL) != None\
    or re.search('.*(uae|emirates).*', entities["urls"][0]["expanded_url"], re.DOTALL) != None:
        urlContentCredibility = True
    else:
        urlContentCredibility = False

urlContentCredibility = False
if qtFlag:
    if len(qtEntities["urls"]) >0:
        if re.search('.*fact-?check.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None\
        or re.search('.*united.*arab.*emirates.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None \
        or re.search('.*700.*(crore|cr).*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None\
        or re.search('.*100.*million.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None \
        or re.search('.*yusuff.*ali.*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None\
        or re.search('.*(uae|emirates).*', qtEntities["urls"][0]["expanded_url"], re.DOTALL) != None:
            urlContentCredibility = True
        else:
            urlContentCredibility = False

if uaeFlag or quaeFlag or urlContentCredibility or urlContentCredibility:
    exeptionFlag = False
else:
    exeptionFlag = True   

In [ ]:
for file in tqdm(os.listdir(adr_kr_3days)):
    qtText, qtEntities, qtUser, rtUser, tweetText, qtTweetText, rtUser = None, None, None, None, None, None, None

    tweet = json.load(open(adr_kr_3days+file))    
    personalViewFlag = True
    
    elements = tweetElements(tweet)
    rtFlag = copy.deepcopy(elements[0])
    qtFlag = copy.deepcopy(elements[1]) 

    
    if rtFlag == False and qtFlag == False:
        text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
    elif rtFlag == True and qtFlag == False:
        text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
    elif rtFlag == False and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
    elif rtFlag == True and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])
                    
    tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
    qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "



uaeFlag = False
if re.search('.*commies*', tweetText, re.DOTALL) != None:    
    uaeFlag = True
else:
    uaeFlag = False

if uaeFlag:
    shutil.move(adr_kr_3days+file, adr_kr_outlets)